In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, GlobalMaxPool1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer

# Load the IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

max_len = 500  
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

# Build the model
model = Sequential()

# Removed input_length argument
model.add(Embedding(input_dim=10000, output_dim=128))  # Removed input_length

model.add(Bidirectional(LSTM(128, return_sequences=True)))

model.add(GlobalMaxPool1D())

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test), verbose=2)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Prepare new reviews for prediction
new_reviews = [
    "The movie was amazing! I loved the acting and the plot was very engaging.",
    "I hated this movie. It was too slow and the characters were boring."
]

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(new_reviews)
new_reviews_sequences = tokenizer.texts_to_sequences(new_reviews)
new_reviews_padded = pad_sequences(new_reviews_sequences, maxlen=max_len)

# Make predictions
predictions = model.predict(new_reviews_padded)
predicted_labels = [1 if p > 0.5 else 0 for p in predictions]

# Output predictions
for review, label in zip(new_reviews, predicted_labels):
    sentiment = "Positive" if label == 1 else "Negative"
    print(f"Review: {review}\nPredicted Sentiment: {sentiment}\n")

Epoch 1/5


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, GlobalMaxPool1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.callbacks import EarlyStopping

# Load the IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

# Set a reduced maximum sequence length
max_len = 300
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

# Create a tf.data dataset for efficient data loading
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=1024).batch(64).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64).prefetch(tf.data.experimental.AUTOTUNE)

# Build the model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128))
model.add(Bidirectional(LSTM(64, return_sequences=True)))  # You can adjust the number of LSTM units
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model
history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=test_dataset,
    callbacks=[early_stopping],
    verbose=2
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset, verbose=0)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

Epoch 1/5
391/391 - 67s - 172ms/step - accuracy: 0.7887 - loss: 0.4255 - val_accuracy: 0.8754 - val_loss: 0.2992
Epoch 2/5
391/391 - 80s - 206ms/step - accuracy: 0.9022 - loss: 0.2453 - val_accuracy: 0.8775 - val_loss: 0.2920
Epoch 3/5
391/391 - 83s - 213ms/step - accuracy: 0.9357 - loss: 0.1713 - val_accuracy: 0.8680 - val_loss: 0.3556
Epoch 4/5
391/391 - 81s - 208ms/step - accuracy: 0.9582 - loss: 0.1161 - val_accuracy: 0.8661 - val_loss: 0.4025
Test Loss: 0.2920, Test Accuracy: 0.8775
